# Pytorch

It is easy to export a Pytorch model to ONNX because it is built into the API. The Pytorch documentation provides a good example on how to perform this conversion.

This is a simplified example:

In [1]:
import numpy as np
import torch
import torchvision

In [2]:
dummy_input = torch.randn(1,3,224,224, device='cpu')
model = torchvision.models.alexnet(pretrained=True)

In [3]:
input_names = ['input_alexnet'] + ['layer_%d' % i for i in range(16)]
output_name = ['output_alexnet'] 

In [4]:
torch.onnx.export(model, dummy_input, 'alexnet_onnx.onnx', verbose=True, input_names= input_names, output_names=output_name)

graph(%input_alexnet : Float(10, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %layer_0 : Float(64, 3, 11, 11, strides=[363, 121, 11, 1], requires_grad=1, device=cpu),
      %layer_1 : Float(64, strides=[1], requires_grad=1, device=cpu),
      %layer_2 : Float(192, 64, 5, 5, strides=[1600, 25, 5, 1], requires_grad=1, device=cpu),
      %layer_3 : Float(192, strides=[1], requires_grad=1, device=cpu),
      %layer_4 : Float(384, 192, 3, 3, strides=[1728, 9, 3, 1], requires_grad=1, device=cpu),
      %layer_5 : Float(384, strides=[1], requires_grad=1, device=cpu),
      %layer_6 : Float(256, 384, 3, 3, strides=[3456, 9, 3, 1], requires_grad=1, device=cpu),
      %layer_7 : Float(256, strides=[1], requires_grad=1, device=cpu),
      %layer_8 : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %layer_9 : Float(256, strides=[1], requires_grad=1, device=cpu),
      %layer_10 : Float(4096, 9216, strides=[9216, 1], requires_gr

# TensorFlow

Exporting a TensorFlow neural network to ONNX takes a bit longer than with Pytorch, but it is still straightforward. 

Tensorflow and ONNX both define their own graph format to represent to model. You can use tensorflow-onnx to export a Tensorflow model to ONNX.

> Procedures to convert tensorflow model

>   - get tensorflow model
>    - convert to ONNX
>    - validate

Tensorflow uses several file formats to represent a model, such as checkpoint files, graph with weight(called frozen graph next) and saved_model, and it has APIs to generate these files. And tensorflow-onnx can accept all the three formats to represent a Tensorflow model, the format "saved_model" should be the preference since it doesn't require the user to specify input and output names of graph.

*Resources:*
https://github.com/onnx/tutorials

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import onnxruntime

x = np.random.randn(1,224,224,3).astype(np.float32)
x = preprocess_input(x)
print(x.dtype)

float32


In [2]:
model = ResNet50(weights='imagenet')
preds = model.predict(x)
print('Keras Predicted:', decode_predictions(preds, top=3)[0])
model.save('resnet_model')

Keras Predicted: [('n03782006', 'monitor', 0.09025134), ('n04404412', 'television', 0.08011744), ('n04286575', 'spotlight', 0.049681667)]
INFO:tensorflow:Assets written to: resnet_model\assets


In [3]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = "resnet_model.onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [5]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": x})

print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])

# make sure ONNX and keras have the same results
np.testing.assert_allclose(preds, onnx_pred[0], rtol=1e-5)

ONNX Predicted: [('n03782006', 'monitor', 0.09025135), ('n04404412', 'television', 0.08011768), ('n04286575', 'spotlight', 0.04968174)]


In [ ]:
# python -m tf2onnx.convert --opset 13 --saved-model {"resnet_model"} --output  {"resnet_model.onnx"}